In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import cv2
import os
import zipfile
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np




In [3]:
zip_file_path = '/content/drive/MyDrive/PerspectAI/dataset_a.zip'

In [4]:
extract_dir='/content/dataset'

In [5]:
with zipfile.ZipFile(zip_file_path,'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [6]:
import os
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch
from google.colab.patches import cv2_imshow

# Define the apply_esrgan function
def apply_esrgan(img, model, device):
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round().astype(np.uint8)

    return output

In [7]:
# Load the ESRGAN model
model_path = '/content/drive/MyDrive/ESRGANmodel/RRDB_ESRGAN_x4.pth'
device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> 'cpu'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)



In [8]:
def detect_faces(image):

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    return faces

In [9]:
datasets=['train_ds','val_ds','test_ds']

train_preprocessed=[]
val_preprocessed=[]
test_preprocessed=[]


In [18]:
!pip3 install tqdm

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Assuming detect_faces and apply_esrgan functions are already defined

datasets = ['train_ds', 'val_ds', 'test_ds']

from google.colab.patches import cv2_imshow

# Initialize lists for storing preprocessed images
train_preprocessed = []
val_preprocessed = []
test_preprocessed = []

train_labels = []
val_labels = []
test_labels = []

def load_images_and_labels(dataset):
    images = []
    labels = []
    dataset_path = os.path.join(extract_dir, dataset)
    people_folders = os.listdir(dataset_path)

    progress_bar = tqdm(people_folders, desc=f'Processing {dataset}', unit='folder')


    for person_folder in people_folders:
         person_path = os.path.join(dataset_path, person_folder)
         image_files = [os.path.join(person_path, f) for f in os.listdir(person_path) if f.endswith('.jpg') or f.endswith('.png')]
         label = person_folder
         for image_path in image_files:
               image = cv2.imread(image_path, cv2.IMREAD_COLOR)
               faces = detect_faces(image)

               for (x, y, w, h) in faces:
                # Extract the face ROI
                  face_roi = image[y:y+h, x:x+w]
                # Apply ESRGAN to the face ROI
                  enhanced_face = apply_esrgan(face_roi, model, device)

                # Convert to grayscale
                  gray_face = cv2.cvtColor(enhanced_face, cv2.COLOR_BGR2GRAY)

                # Resize the face to the target size
                  target_size = (64, 64)
                  resized_face = cv2.resize(gray_face, target_size)

                  images.append(resized_face)
                  labels.append(label)



    return images,labels

train_preprocessed, train_labels = load_images_and_labels('train_ds')
val_preprocessed, val_labels = load_images_and_labels('val_ds')
test_preprocessed, test_labels = load_images_and_labels('test_ds')

print("Image loading and enhancement completed!")




Processing train_ds:   0%|          | 0/1755 [00:00<?, ?folder/s]

In [ ]:
import torch

# Check if CUDA (GPU acceleration) is available
cuda_available = torch.cuda.is_available()

if cuda_available:
    # Get the CUDA device count
    cuda_device_count = torch.cuda.device_count()
    print('CUDA is available with {cuda_device_count} CUDA device(s)!')
else:
    print('CUDA is not available. Running on CPU.')


CUDA is available with {cuda_device_count} CUDA device(s)!


In [ ]:
from google.colab.patches import cv2_imshow


In [ ]:
def compute_covmatrix(images):
  num_images=len(images)
  mean_images=np.mean(images,axis=0)
  cov_matrix=np.zeros((mean_image.shape[0],mean_image.shape[0]))

  for img in images:
    diff=img-mean_image
    cov_matrix+=np.dot(diff.T,diff)

  cov_matrix /= num_images
  return cov_matrix


train_cov_matrix = compute_covmatrix(train_preprocessed)
val_cov_matrix = compute_covmatrix(val_preprocessed)
test_cov_matrix = compute_covmatrix(test_preprocessed)

NameError: name 'np' is not defined

In [ ]:
def extract_principal_components(cov_matrix,num_components):
  eig_vals,eig_vecs=np.linalg.eigh(cov_matrix)

  sorted_indices=np.argsort(eig_vals)[::-1]
  sorted_eig_vals=eig_vals[sorted_indices]
  sorted_eig_vecs=eig_vecs[:,sorted_indices]


  principal_components=sorted_eig_vec[:,:num_components]

  return principal_components


num_components = 50

train_principal_components = extract_principal_components(train_cov_matrix, num_components)
val_principal_components = extract_principal_components(val_cov_matrix, num_components)
test_principal_components = extract_principal_components(test_cov_matrix, num_components)

NameError: name 'train_cov_matrix' is not defined

In [ ]:
def project_images(images,principal_components):
    num_images = len(images)
    flattened_images = images.reshape(num_images, -1)  # Flatten each image into a vector
    projected_images = np.dot(flattened_images, principal_components)

    return projected_images

train_projected = project_images(train_preprocessed, train_principal_components)


NameError: name 'train_principal_components' is not defined

In [ ]:
!pip3 install sklearn
from sklearn.neighbors import KNeighborsClassifier

def train_classifier(projections,labels):
  knn=KNeighborsClassifier(n_neighbors=5)
  knn.fit(projections,labels)
  return knn


knn_classifier = train_classifier(train_projected, train_labels)


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


NameError: name 'train_projected' is not defined

In [ ]:
from sklearn.metrics import accuracy_score, classification_report


def evaluate_classifier(knn_classifier, test_images, test_labels):
    # Project test images onto principal components

    num_images=len(test_images)
    flattened_images = test_images.reshape(num_images, -1)
    projected_images = np.dot(flattened_images, knn_classifier.components_.T)


    # Predict labels for test images
    predictions = knn_classifier.predict(test_projections)

    # Calculate accuracy
    accuracy = accuracy_score(test_labels, predictions)

    # Generate classification report (optional)
    report = classification_report(test_labels, predictions)

    return accuracy, report

# Evaluate classifier on test dataset
val_accuracy, val_report = evaluate_classifier(knn_classifier, val_preprocessed, val_labels)

print("Validation Accuracy: {val_accuracy}")
print("Classification Report:\n", val_report)

NameError: name 'knn' is not defined

In [ ]:
def recognize_faces(classifier, images, labels):
    # Flatten and project test images
    num_images = len(images)
    flattened_images = images.reshape(num_images, -1)
    projected_images = np.dot(flattened_images, classifier.components_.T)

    # Predict labels for test images
    predictions = classifier.predict(projected_images)

    return predictions

# Perform face recognition on test set
test_predictions = recognize_faces(knn_classifier, test_preprocessed, test_labels)
